In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import leafmap.foliumap as leafmap
import time
import requests
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
import os
import re

In [ ]:
path_row = "data/fiber_lines.gpkg"
path_customers = "data/coverage_streets_raw.gpkg"

Load layers in `Right of Way` file

In [ ]:
# Load all layers from right_of_way.gpkg
layers = fiona.listlayers(path_row)
gdf = [gpd.read_file(path_row, layer=lyr) for lyr in layers]
layers

Merge all the layers in the `Right of Way` package

In [ ]:
# Merge them into one GeoDataFrame
gdf_right = gpd.GeoDataFrame(pd.concat(gdf, ignore_index=True), crs=gdf[0].crs)

Read `Customer coverage` data

In [ ]:
# Load customer coverage lines
gdf_all = gpd.read_file(path_customers)

# Make sure they’re in the same CRS
gdf_all = gdf_all.to_crs(gdf_right.crs)

- Right of Way has 699 LineStrings
- Total Customer Coverage has 4522 LineStrings

In [ ]:
gdf_all["geometry"]

Break down MultiLineStrings into LineStrings so that the lines that can iterated individually

In [ ]:
gdf_right = gdf_right.explode(index_parts=False, ignore_index=True)
gdf_all = gdf_all.explode(index_parts=False, ignore_index=True)

Check for lines that intersect between the Right of Way and Customer Coverage and filter them out
- The lines may be really close and not perfectly intersect, so we need a buffer for 5 metres

In [ ]:
# Option 1: exact geometry comparison

buffered_right = gdf_right.buffer(39)  # 5 meters tolerance
# gdf_non_right = gdf_all[~gdf_all.intersects(buffered_right.union_all())]
gdf_non_right = gdf_all[~gdf_all.intersects(gdf_right.unary_union)]
gdf_non_right["geometry"].value_counts().sum()

There are 2967 LineStrings without Right of Way
- 4522 -> 2967 -> 2844(buffer of 5m) -> 2594(buffer off 39m)

Calculate distance and assign values to a field

In [ ]:
gdf_non_right = gdf_non_right.to_crs(epsg=32631)
gdf_non_right["distance_m"] = gdf_non_right.geometry.length

In [ ]:
gdf_non_right

Plot the Customer Coverage without Right of Way

In [ ]:
# Create a map centered roughly on your data
m = leafmap.Map(center=[6.45, 3.39], zoom=9, style="streets")

# Style for non-right-of-way lines
style_non_right = {
    "color": "blue",
    "weight": 2,
}

# Style for right-of-way lines
style_right = {
    "color": "red",
    "weight": 2,
}

# Add the GeoDataFrame
# tooltip_fields = [col for col in ["road_street_name", "LOCAL GOVERNMENT", "distance(m)"] if col in gdf_non_right.columns]

# Add non-right-of-way roads
m.add_gdf(
    gdf_non_right,
    layer_type="line",
    layer_name="Non-Right-of-Way Roads",
    style=style_non_right,
)

# Add right-of-way roads
m.add_gdf(
    gdf_right,
    layer_type="line",
    layer_name="Right-of-Way Roads",
    style=style_right,
)

# Zoom to fit both datasets
m.zoom_to_gdf(pd.concat([gdf_non_right, gdf_right]))

m

Browser

In [ ]:
# webbrowser.open("map.html")

Save File

In [ ]:
#

Reverse geocode lines to get coordinates

In [ ]:
API_KEY = os.getenv("banjo_google_api_key")

In [ ]:
geolocator = GoogleV3(api_key=API_KEY, timeout=10)
# Set to False to raise exceptions on failures (useful for debugging)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5, swallow_exceptions=False)
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=5, swallow_exceptions=False )

In [ ]:
def reverse_geocode(lat, lon, api_key):
    """
    Reverse geocode coordinates using Google Maps API.
    Returns the formatted address, or None if not found.
    """
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
        response = requests.get(url)
        result = response.json()

        if result["status"] == "OK" and len(result["results"]) > 0:
            address = result["results"][0]["formatted_address"]
            print(f"Address found: {address}")
            return address
        else:
            print(f"No address found for {lat}, {lon}")
            return None
    except Exception as e:
        print(f"Error reverse geocoding ({lat}, {lon}): {e}")
        return None

In [ ]:
def batch_reverse_geocode(gdf, api_key, batch_size=50, delay=2):
    """
    For each LineString in the GeoDataFrame, compute centroid,
    reverse geocode it with Google API, and add results as new columns.
    """
    # Ensure centroids are computed
    gdf["centroid"] = gdf.geometry.centroid
    gdf["lat"] = gdf.centroid.y
    gdf["lon"] = gdf.centroid.x

    addresses = []

    for i, row in gdf.iterrows():
        lat, lon = row["lat"], row["lon"]
        address = reverse_geocode(lat, lon, api_key)  # <- your function
        addresses.append(address)

        # Pause after a batch to avoid quota issues
        if (i + 1) % batch_size == 0:
            print(f"Processed {i+1} rows, pausing {delay}s...")
            time.sleep(delay)

    gdf["address"] = addresses
    return gdf

Test Run

In [ ]:
gdf_sample = gdf_non_right.head(40).copy()
if gdf_sample.crs != "EPSG:4326":
    gdf_sample = gdf_sample.to_crs(epsg=4326)

gdf_non_right_with_addr = batch_reverse_geocode(gdf=gdf_sample, api_key=API_KEY)

In [ ]:
def extract_street_name(address: str) -> str:
    """
    Extract street name from a Google Maps formatted address.
    Examples:
        "9 Abudu Oladejo St, Papa Ashafa, Lagos 102212, Lagos, Nigeria"
            -> "Abudu Oladejo St"
        "13b Peace Cl, Ifako Agege, Lagos 101232, Lagos, Nigeria"
            -> "Peace Cl"
        "11A Ajayi Rd, Ojodu, Ikeja 300001, Lagos, Nigeria"
            -> "Ajayi Rd"
        "Harmony Estate, 29 Kolapo Boluwade Cres, Ifako-Ijaiye, Lagos 101232, Lagos, Nigeria"
            -> "Kolapo Boluwade Cres"   # (if estate name comes first, we still extract street)
    """

    if not isinstance(address, str) or not address.strip():
        return ""

    # Split by comma, take the first chunk (usually house no. + street)
    first_part = address.split(",")[0].strip()

    # Remove house numbers (e.g., "18 " or "13b ")
    first_part = re.sub(r"^\d+[A-Za-z\-]*\s+", "", first_part)

    # Return cleaned street part
    return first_part.strip()

In [ ]:
gdf_sample["name"] = gdf_sample["name"].fillna(gdf_sample["address"].apply(extract_street_name))

In [ ]:
gdf_sample

Plot Sample

In [ ]:
m = leafmap.Map(center=[6.45, 3.39], zoom=9, style="streets")
gdf_plot = gdf_sample.drop(columns=["centroid", "id", "code", "ref", "rid"], errors="ignore")
gdf_plot.to_crs(epsg=32631)
m.add_gdf(
    gdf_plot,
    layer_type="line",
    layer_name="Non-Right-of-Way Roads",
    style={
    "color": "blue",
    "weight": 2,
},
)
m

Convert from UTM to WGS84 for compatibility with Google API

In [ ]:
if gdf_non_right.crs != "EPSG:4326":
    gdf_non_right = gdf_non_right.to_crs(epsg=4326)

gdf_non_right_with_addr = batch_reverse_geocode(gdf=gdf_non_right, api_key=API_KEY)

In [ ]:
from difference_intersecting_lines import filter_non_intersecting_lines

gdf = filter_non_intersecting_lines("data/fiber_lines.gpkg", "data/coverage_streets_raw.gpkg", api_key=API_KEY)

In [ ]:
# 6.476630356790446, 3.6095868468658163 is Karimu Adeyi Street, Lekki, Lagos, Nigeria
file = gdf["gdf"]
# file.to_file("customers_without_right_of_way.gpkg")
# file.to_csv("customers_without_right_of_way.csv", index=False)